In [12]:
from StartingCandidateGenerator import StartingCandidateGenerator as SCG
from DataBaseInterface import DataBaseInterface as DBI
from CrossOperation import CrossOperation as CO
"Import necessary to fix atoms in the slab"
from ase.constraints import FixAtoms
from ase import Atoms
from ase.io import read
from generate_ceo2 import create_ceo2_111 as ceo2Slab
import numpy as np
from ase.visualize import view
import numpy as np
from ase.optimize import BFGS
from ase.calculators.emt import EMT

#--------------------------Define the fitness fucntion for your atoms-----------------------"

def fitness_function(atoms,env,reference = 0.0,au_energy = 0.0)-> float:
    o2_energy = read('test_data/o2_pw91.traj').get_potential_energy()
    reference=0.0# read('test_data/pt6o0_fs_meta.traj@:')[0].get_potential_energy()

    at_num= atoms.get_atomic_numbers()
    pt_num = np.count_nonzero(at_num == 78)
    au_num= np.count_nonzero(at_num == 79)

    fre = atoms.get_potential_energy() - reference - au_num * (au_energy) - au_num*env
    return fre

#---------------------------Generate static part of the system------------------------------"
slab = ceo2Slab(number_of_layers=1,repetitions = (5,5,1),vacuum=12.0)
a = 12.0
slab = Atoms(cell=[a,a,a],
             pbc=True)
c = FixAtoms(indices=np.arange(len(slab)))
slab.set_constraint(c)
#If slab atoms arent fixed they will be generated  for all structures, but will be relaxed!"

#---------------------------Generate constant part of the system------------------------------"
#Part of the system that can be relaxed but that does not vary in number over the duration of the search"
constant = Atoms('Pt3')

#---------Generate variable part of the system-(Single atom types only)------------------"
#Part of the system that can be relaxed and that varyies in number over the duration of the search"
#
variable_type = Atoms('Au')
#Considered number of variable type atoms n the search"
variable_range = [0,1,2,3,4,5,6,7,8,9,10]

#---------------------------Define starting population--------------------------------"

candidateGenerator = SCG(slab,constant,variable_type,variable_range)
crossing = CO(slab,constant,variable_type,variable_range,stc_change_chance = 0.5)
db = DBI('databaseGA.db')
population = 25

starting_pop = candidateGenerator.get_starting_population(population_size=population)

#-----------------------------------Define Calculator----------------------------------------"

calc = EMT()

#Get Reference energies##########
constant.cell = slab.get_cell()
variable_type.cell = slab.get_cell()

constant.calc = calc
dyn = BFGS(constant)
dyn.run(steps=100, fmax=0.05)
ref = constant.get_potential_energy()
variable_type.calc = calc
dyn = BFGS(variable_type)
dyn.run(steps=100, fmax=0.05)
au_en = variable_type.get_potential_energy()
#####################################



for i in starting_pop:
    db.add_unrelaxed_candidate(i )

#---------------------------------Relax initial structures-----------------------------------"
env = -0.5 #Environment chem pot 
while db.get_number_of_unrelaxed_candidates() > 0:

    atoms = db.get_first_unrelaxed()
    
    atoms.calc = calc
    dyn = BFGS(atoms)
    dyn.run(steps=100, fmax=0.05)
    atoms.get_potential_energy()
    
    atoms.info['key_value_pairs']['raw_score'] = -fitness_function(atoms,env,reference = ref, au_energy = au_en)
   
    db.update_to_relaxed(atoms.info['key_value_pairs']['dbid'],atoms)

#--------------------------------------Find better stoich to srtart eval----------------------------
#Overall fitness value
steps = 100
sub_steps = 20
for i in range(steps):
    for j in range(sub_steps):
        atomslist = db.get_better_candidates(n=10)
        #Choose two of the most stable structures to pair
        cand1 = np.random.randint(0,10)
        cand2 = np.random.randint(0,10)
        while cand1 == cand2:
            cand2 = np.random.randint(0,10)
            
        #Mate the particles
        res  = crossing.cross(atomslist[cand1],atomslist[cand2])
        if(res is not None):
            if(len(atomslist[cand1])!= len(res) and len(atomslist[cand2]) != len(res)):
                print("--------------STCCHANGE-----------------")
            db.add_unrelaxed_candidate(res)
            
    
    while db.get_number_of_unrelaxed_candidates() > 0:

        atoms = db.get_first_unrelaxed()

        atoms.calc = calc
        dyn = BFGS(atoms)
        dyn.run(steps=100, fmax=0.05)
        atoms.get_potential_energy()

        atoms.info['key_value_pairs']['raw_score'] = -fitness_function(atoms,env,reference = ref, au_energy = au_en)

        db.update_to_relaxed(atoms.info['key_value_pairs']['dbid'],atoms)
        
atomslist = db.get_better_candidates(n=1000)
for at in atomslist:
    print(at.numbers)
view(atomslist[0])

      Step     Time          Energy         fmax
BFGS:    0 14:18:17    21203.444554           nan
BFGS:    1 14:18:17       17.550000        0.0000
      Step     Time          Energy         fmax
BFGS:    0 14:18:17        3.800000        0.0000
      Step     Time          Energy         fmax
BFGS:    0 14:18:18       12.551329        6.1899
BFGS:    1 14:18:18       11.524749        6.9387
BFGS:    2 14:18:18        9.190784        4.5432
BFGS:    3 14:18:18       10.771983       19.3960
BFGS:    4 14:18:18        8.941603        2.6773
BFGS:    5 14:18:18        8.866860        1.4743
BFGS:    6 14:18:18        8.820840        1.2228
BFGS:    7 14:18:18        8.798689        1.3296
BFGS:    8 14:18:18        8.428290        1.4634
BFGS:    9 14:18:18        7.824160        2.3952
BFGS:   10 14:18:18        6.885763        3.5436
BFGS:   11 14:18:18        5.660298        3.9068
BFGS:   12 14:18:18        4.993645        2.0663
BFGS:   13 14:18:18        4.981830        1.2348
BFG

BFGS:   10 14:18:18        8.413802        3.2939
BFGS:   11 14:18:18        8.335319        1.7656
BFGS:   12 14:18:18        8.287972        1.2977
BFGS:   13 14:18:18        8.252799        0.8041
BFGS:   14 14:18:18        8.228525        0.1649
BFGS:   15 14:18:18        8.226035        0.0583
BFGS:   16 14:18:19        8.225768        0.0386
      Step     Time          Energy         fmax
BFGS:    0 14:18:19       20.875107        3.4233
BFGS:    1 14:18:19       19.970548        3.4786
BFGS:    2 14:18:19       16.210232        5.0507
BFGS:    3 14:18:19       13.673692       10.4345
BFGS:    4 14:18:19       12.032855       15.1096
BFGS:    5 14:18:19       10.036734        4.8747
BFGS:    6 14:18:19        8.966199        2.6475
BFGS:    7 14:18:19        9.140315        7.1759
BFGS:    8 14:18:19        8.567545        1.1177
BFGS:    9 14:18:19        8.485156        1.0246
BFGS:   10 14:18:19        8.349936        1.2519
BFGS:   11 14:18:19        8.290793        1.1152
B

BFGS:   54 14:18:19        6.851025        0.3603
BFGS:   55 14:18:19        6.834312        0.6141
BFGS:   56 14:18:19        6.820687        0.7083
BFGS:   57 14:18:19        6.784713        0.5422
BFGS:   58 14:18:19        6.759811        0.3085
BFGS:   59 14:18:19        6.754774        0.2912
BFGS:   60 14:18:19        6.751868        0.2295
BFGS:   61 14:18:19        6.749143        0.1389
BFGS:   62 14:18:19        6.746321        0.0692
BFGS:   63 14:18:19        6.745189        0.0536
BFGS:   64 14:18:19        6.744771        0.0280
      Step     Time          Energy         fmax
BFGS:    0 14:18:19       21.172662        3.4735
BFGS:    1 14:18:19       20.508635        3.7150
BFGS:    2 14:18:19       17.884478        4.3761
BFGS:    3 14:18:19       15.267226        4.2795
BFGS:    4 14:18:19       13.848978        5.5792
BFGS:    5 14:18:19       12.959863        3.4888
BFGS:    6 14:18:19       11.836823        3.9803
BFGS:    7 14:18:19       10.665499        3.4511
B

BFGS:   50 14:18:20        7.548759        0.7441
BFGS:   51 14:18:20        7.508532        0.5801
BFGS:   52 14:18:20        7.486636        0.6552
BFGS:   53 14:18:20        7.459894        0.4871
BFGS:   54 14:18:20        7.437542        0.2467
BFGS:   55 14:18:20        7.428491        0.2182
BFGS:   56 14:18:20        7.422697        0.2037
BFGS:   57 14:18:20        7.419546        0.1746
BFGS:   58 14:18:20        7.416740        0.1322
BFGS:   59 14:18:20        7.414485        0.1302
BFGS:   60 14:18:20        7.413519        0.1260
BFGS:   61 14:18:20        7.411409        0.1140
BFGS:   62 14:18:20        7.409789        0.1346
BFGS:   63 14:18:20        7.407145        0.2209
BFGS:   64 14:18:20        7.401873        0.3594
BFGS:   65 14:18:20        7.381436        0.6793
BFGS:   66 14:18:20        7.362169        1.0839
BFGS:   67 14:18:20        7.339756        0.8760
BFGS:   68 14:18:20        7.343947        0.9364
BFGS:   69 14:18:20        7.301542        1.0626


BFGS:   10 14:18:20       11.331648        1.2214
BFGS:   11 14:18:20       11.172144        1.0528
BFGS:   12 14:18:20       11.101852        1.2032
BFGS:   13 14:18:20       10.841900        2.1236
BFGS:   14 14:18:20       10.751207        1.2487
BFGS:   15 14:18:20       10.585910        0.9367
BFGS:   16 14:18:20       10.500687        0.8921
BFGS:   17 14:18:20       10.461955        0.6877
BFGS:   18 14:18:20       10.430768        0.4329
BFGS:   19 14:18:20       10.424195        0.3659
BFGS:   20 14:18:20       10.413750        0.2949
BFGS:   21 14:18:20       10.406449        0.2902
BFGS:   22 14:18:20       10.395984        0.4298
BFGS:   23 14:18:20       10.364910        0.7682
BFGS:   24 14:18:20       10.284462        1.0878
BFGS:   25 14:18:20       10.148178        0.9549
BFGS:   26 14:18:20       10.465572        4.4944
BFGS:   27 14:18:20       10.052172        0.9803
BFGS:   28 14:18:20        9.995971        0.8657
BFGS:   29 14:18:20        9.827934        1.1781


...

BFGS:   10 14:58:40        9.408886        0.3520
BFGS:   11 14:58:40        9.394076        0.2933
BFGS:   12 14:58:40        9.382203        0.3127
BFGS:   13 14:58:40        9.370080        0.3113
BFGS:   14 14:58:40        9.355501        0.4444
BFGS:   15 14:58:40        9.337096        0.5176
BFGS:   16 14:58:40        9.321289        0.3471
BFGS:   17 14:58:40        9.307362        0.3310
BFGS:   18 14:58:40        9.290289        0.3910
BFGS:   19 14:58:40        9.274812        0.4964
BFGS:   20 14:58:40        9.263501        0.3486
BFGS:   21 14:58:40        9.257177        0.2293
BFGS:   22 14:58:40        9.253626        0.2029
BFGS:   23 14:58:40        9.248535        0.2909
BFGS:   24 14:58:40        9.241539        0.3408
BFGS:   25 14:58:40        9.233949        0.2344
BFGS:   26 14:58:40        9.226301        0.2738
BFGS:   27 14:58:40        9.217131        0.2786
BFGS:   28 14:58:40        9.204392        0.4687
BFGS:   29 14:58:40        9.188801        0.5012


BFGS:   36 14:58:41        8.973507        0.6762
BFGS:   37 14:58:41        8.908903        0.6578
BFGS:   38 14:58:41        8.869961        0.5850
BFGS:   39 14:58:41        8.825551        0.6387
BFGS:   40 14:58:41        8.759845        0.6227
BFGS:   41 14:58:41        8.728530        0.5121
BFGS:   42 14:58:41        8.690898        0.5208
BFGS:   43 14:58:41        8.650286        0.5237
BFGS:   44 14:58:41        8.597452        0.6557
BFGS:   45 14:58:41        8.555476        0.6773
BFGS:   46 14:58:41        8.527294        0.5688
BFGS:   47 14:58:41        8.506407        0.3403
BFGS:   48 14:58:41        8.499319        0.3115
BFGS:   49 14:58:41        8.485157        0.2930
BFGS:   50 14:58:41        8.478545        0.2032
BFGS:   51 14:58:41        8.473514        0.1310
BFGS:   52 14:58:41        8.471390        0.1094
BFGS:   53 14:58:41        8.469290        0.1313
BFGS:   54 14:58:41        8.467397        0.1207
BFGS:   55 14:58:41        8.465673        0.1331


BFGS:   20 14:58:42        7.432616        0.5112
BFGS:   21 14:58:42        7.407431        0.5258
BFGS:   22 14:58:42        7.371380        0.7637
BFGS:   23 14:58:42        7.357769        0.8104
BFGS:   24 14:58:42        7.354093        0.8711
BFGS:   25 14:58:42        7.308373        0.3252
BFGS:   26 14:58:42        7.294051        0.2799
BFGS:   27 14:58:42        7.281798        0.2690
BFGS:   28 14:58:42        7.270850        0.2918
BFGS:   29 14:58:42        7.259404        0.2348
BFGS:   30 14:58:42        7.249991        0.2516
BFGS:   31 14:58:42        7.241749        0.2495
BFGS:   32 14:58:42        7.233373        0.2561
BFGS:   33 14:58:42        7.224789        0.2782
BFGS:   34 14:58:42        7.216680        0.2785
BFGS:   35 14:58:42        7.209134        0.3072
BFGS:   36 14:58:42        7.201531        0.2524
BFGS:   37 14:58:42        7.194821        0.1940
BFGS:   38 14:58:42        7.189348        0.1868
BFGS:   39 14:58:42        7.185168        0.1345


BFGS:   25 14:58:43        9.013648        1.2887
BFGS:   26 14:58:43        8.902531        1.6093
BFGS:   27 14:58:43        8.808890        0.5446
BFGS:   28 14:58:43        8.773884        0.5444
BFGS:   29 14:58:43        8.729728        0.4319
BFGS:   30 14:58:43        8.705920        0.3534
BFGS:   31 14:58:43        8.685092        0.2734
BFGS:   32 14:58:43        8.675295        0.2318
BFGS:   33 14:58:43        8.666826        0.2143
BFGS:   34 14:58:43        8.654783        0.2264
BFGS:   35 14:58:43        8.645591        0.3103
BFGS:   36 14:58:43        8.636240        0.3287
BFGS:   37 14:58:43        8.628479        0.2819
BFGS:   38 14:58:43        8.619875        0.2455
BFGS:   39 14:58:43        8.607874        0.2677
BFGS:   40 14:58:43        8.595294        0.3034
BFGS:   41 14:58:43        8.583428        0.3072
BFGS:   42 14:58:43        8.571743        0.2881
BFGS:   43 14:58:43        8.558496        0.2999
BFGS:   44 14:58:43        8.536885        0.3667


<Popen: returncode: None args: ['/home/jon/Documents/GrandCanonicalGA/ase_en...>